In [2]:
import os, glob, random, pathlib, pprint

DATA_DIR = pathlib.Path("/kaggle/input/data-set-s/MOT20Det")  # same as before

# Look only inside the *train* split where images + gt are guaranteed
train_seqs = sorted([p for p in (DATA_DIR/'train').iterdir() if p.is_dir()])
print("Train sequences:", [p.name for p in train_seqs])

# Pick the first sequence that actually contains frames
sample_img = None
for seq in train_seqs:
    imgs = glob.glob(f"{seq}/img1/*.jpg") or glob.glob(f"{seq}/img1/*.png")
    if imgs:
        sample_img = random.choice(imgs)
        gt_file   = seq/'gt/gt.txt'
        break

print("Example frame file:", sample_img)
print("Ground-truth file :", gt_file)


Train sequences: ['MOT20-01', 'MOT20-02', 'MOT20-03', 'MOT20-05']
Example frame file: /kaggle/input/data-set-s/MOT20Det/train/MOT20-01/img1/000155.jpg
Ground-truth file : /kaggle/input/data-set-s/MOT20Det/train/MOT20-01/gt/gt.txt


In [3]:
import shutil, pandas as pd, cv2, pathlib, tqdm, glob

SRC_ROOT = pathlib.Path("/kaggle/input/data-set-s/MOT20Det/train")   # read-only
DST_ROOT = pathlib.Path("/kaggle/working/MOT20Det/train")           # writeable
DST_ROOT.mkdir(parents=True, exist_ok=True)

# 1. Copy every sequence so we have write access
for seq in SRC_ROOT.iterdir():
    if not seq.is_dir(): 
        continue
    dst_seq = DST_ROOT/seq.name
    if not dst_seq.exists():                     # skip if you already copied once
        print(f"Copying {seq.name} → working...")
        shutil.copytree(seq, dst_seq)            # 2–3 min

# 2. Create YOLO label files next to the copied images
SEQ_DIRS = sorted([p for p in DST_ROOT.iterdir() if p.is_dir()])

for seq_dir in tqdm.tqdm(SEQ_DIRS, desc="Label-conversion"):
    gt_path = seq_dir / "gt" / "gt.txt"
    gt = pd.read_csv(gt_path, header=None)
    gt.columns = ["frame","tid","x","y","w","h","conf","cls","vis"]

    # get resolution from first frame
    first_img = next((seq_dir/"img1").glob("*.jpg"))
    H, W, _ = cv2.imread(str(first_img)).shape

    for f_idx, rows in gt.groupby("frame"):
        label_path = seq_dir / "img1" / f"{int(f_idx):06d}.txt"
        with open(label_path, "w") as f:
            for _, r in rows.iterrows():
                cx = (r.x + r.w/2) / W
                cy = (r.y + r.h/2) / H
                bw = r.w / W
                bh = r.h / H
                f.write(f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")


Copying MOT20-02 → working...
Copying MOT20-03 → working...
Copying MOT20-01 → working...
Copying MOT20-05 → working...


Label-conversion: 100%|██████████| 4/4 [01:11<00:00, 17.84s/it]


In [4]:
# ------------------------------------------------------------
# 1. Define split and make symlinks  (≈ 1 second, no disk copy)
# ------------------------------------------------------------
import os, pathlib, subprocess

ROOT = pathlib.Path("/kaggle/working/MOT20Det/train")   # where the sequences were copied
SPLIT_ROOT = pathlib.Path("/kaggle/working/mot20_split")
(SPLIT_ROOT / "train").mkdir(parents=True, exist_ok=True)
(SPLIT_ROOT / "val").mkdir(parents=True, exist_ok=True)

TRAIN_SEQS = ["MOT20-01", "MOT20-03"]
VAL_SEQS   = ["MOT20-02"]          # keep MOT20-04 unseen for later testing

def safe_symlink(src, dst):
    try:
        os.symlink(src, dst)
    except FileExistsError:
        pass

for seq in TRAIN_SEQS:
    safe_symlink(ROOT/seq, SPLIT_ROOT/"train"/seq)
for seq in VAL_SEQS:
    safe_symlink(ROOT/seq, SPLIT_ROOT/"val"/seq)

print("Symlinks done:")
print("  train →", list((SPLIT_ROOT/"train").iterdir()))
print("  val   →", list((SPLIT_ROOT/"val").iterdir()))


Symlinks done:
  train → [PosixPath('/kaggle/working/mot20_split/train/MOT20-03'), PosixPath('/kaggle/working/mot20_split/train/MOT20-01')]
  val   → [PosixPath('/kaggle/working/mot20_split/val/MOT20-02')]


In [5]:
%%writefile mot20.yaml
path: /kaggle/working/mot20_split      # DO NOT end with '/'
train: train
val: val
nc: 1
names: ["person"]


Writing mot20.yaml


In [8]:
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.2/yolov9s.pt -O yolov9s.pt
!ls -lh yolov9s.pt     # quick check: file size ~ 27 MB


-rw-r--r-- 1 root root 0 Apr 28 17:05 yolov9s.pt


In [15]:
# delete the broken file if it exists
!rm -f yolov9s.pt

# use curl -L (follows redirects) and retry if the file is < 10 MB
!curl -L -o yolov9s.pt https://github.com/WongKinYiu/yolov9/releases/download/v0.2/yolov9s.pt
!ls -lh yolov9s.pt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     9  100     9    0     0    100      0 --:--:-- --:--:-- --:--:--   101
-rw-r--r-- 1 root root 9 Apr 28 17:11 yolov9s.pt


In [15]:
!pip install -q --upgrade ultralytics   # pulls the stable 8.x series


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 whic

PREMANI

In [16]:
import pathlib, sys, os

MODEL_DIR = pathlib.Path("/kaggle/input/yolov8n/pytorch/demo/1")
pt_files   = sorted(MODEL_DIR.rglob("*.pt"))
assert pt_files, f"No .pt file found under {MODEL_DIR}"
WEIGHT_PATH = str(pt_files[0])

print("Using weight:", WEIGHT_PATH)
print("Size:", os.path.getsize(WEIGHT_PATH)/1e6, "MB")


Using weight: /kaggle/input/yolov8n/pytorch/demo/1/yolov8n.pt
Size: 6.200696 MB


In [17]:
WEIGHT_PATH="/kaggle/input/yolov8n/pytorch/demo/1/yolov8n.pt"

In [ ]:
!yolo train task=detect model=/kaggle/input/yolov8n/pytorch/demo/1/yolov8n.pt data=mot20.yaml epochs=40 imgsz=800 batch=8 mosaic=1.0 close_mosaic=10 device=0 workers=4 project=mot20_yolov8n_user name=exp


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.119 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/yolov8n/pytorch/demo/1/yolov8n.pt, data=mot20.yaml, epochs=40, time=None, patience=100, batch=8, imgsz=800, save=True, save_period=-1, cache=False, device=0, workers=4, project=mot20_yolov8n_user, name=exp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.

In [ ]:
!ls -lh /kaggle/working/runs/detect/mot20_yolov8n_user/exp/weights


In [1]:
print("hello ")

hello 


2 am

In [4]:
%pip install -q "ultralytics>=8.3.117" cython filterpy scikit-image \
               opencv-python-headless==4.10.0.84 tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following de

In [5]:
import sys, torch, ultralytics as ul
print("Py:", sys.version)
print("Torch:", torch.__version__)
print("Ultralytics:", ul.__version__)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Py: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Torch: 2.5.1+cu124
Ultralytics: 8.3.119


In [6]:
from ultralytics import YOLO
import cv2, torch, time, pathlib, collections, numpy as np

# MOT20 root that you uploaded
DATA_ROOT = pathlib.Path("/kaggle/input/data-set-s/MOT20Det")

# choose one sequence to demo (change ‘MOT20-01’ ⇄ 02/03/04 when you like)
SEQ_DIR = DATA_ROOT / "train/MOT20-01/img1"

# inference device
DEVICE = 0 if torch.cuda.is_available() else "cpu"


In [7]:
model = YOLO("yolov8n.pt")   # start with nano; upgrade to yolov8m/l if you still hit 30 FPS
model.fuse()
if DEVICE != "cpu":
    model.model.half()       # half-precision only on GPU


100%|██████████| 6.25M/6.25M [00:00<00:00, 86.2MB/s]


YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


In [9]:
class LiveInventory:
    """Track IDs across frames and raise NEW / MISSING events."""
    def __init__(self, miss_tolerance=30):
        self.last_seen   = {}          # id → frame_idx
        self.frame_idx   = 0
        self.miss_tol    = miss_tolerance

    def update(self, tracks):
        present_ids = {int(t.id) for t in tracks if t.id is not None}

        # NEW
        new_ids = [tid for tid in present_ids if tid not in self.last_seen]

        # refresh time-stamps
        for tid in present_ids:
            self.last_seen[tid] = self.frame_idx

        # MISSING
        missing_ids = [tid for tid, last in list(self.last_seen.items())
                       if self.frame_idx - last > self.miss_tol]
        for tid in missing_ids:
            del self.last_seen[tid]

        self.frame_idx += 1
        return new_ids, missing_ids


In [10]:
def stream_imgs(seq_dir):
    for img_path in sorted(seq_dir.glob("*.jpg")):
        yield cv2.imread(str(img_path)), img_path.name


In [13]:
def run_sequence(seq_dir, show=False):
    inv  = LiveInventory(miss_tolerance=30)   # ~1 s at 30 FPS
    t0   = time.time()

    for frame, fname in stream_imgs(seq_dir):
        # Ultralytics track API; persist=True keeps ID assignment continuous
        results = model.track(frame,
                              tracker="bytetrack.yaml",
                              persist=True,
                              imgsz=800,
                              device=DEVICE,
                              verbose=False)

        tracks = results[0].boxes

        new_ids, missing_ids = inv.update(tracks)

        # log events
        if new_ids:
            print(f"[{inv.frame_idx:05}] NEW → {new_ids}")
        if missing_ids:
            print(f"[{inv.frame_idx:05}] MISSING → {missing_ids}")

        # optional preview (avoid VideoWriter for max FPS)
        if show:
            vis = results[0].plot()
            cv2.putText(vis,
                        f"NEW:{len(new_ids)}  MISS:{len(missing_ids)}",
                        (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,255,0),2)
            cv2.imshow("live", vis)
            if cv2.waitKey(1) == 27: break  # Esc quits

    dt = time.time() - t0
    print(f"\nProcessed {inv.frame_idx} frames in {dt:.1f}s  →  {inv.frame_idx/dt:.2f} FPS")


In [14]:
run_sequence(SEQ_DIR, show=False)   # set show=True if you launch with a GUI (not in Kaggle)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.8 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.0s, installed 1 package: ['lap>=0.5.12']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

[00001] NEW → [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[00004] NEW → [22]
[00005] NEW → [25]
[00007] NEW → [31]
[00009] NEW → [34]
[00016] NEW → [48]
[00017] NEW → [51]
[00018] NEW → [53]
[00019] NEW → [55]
[00020] NEW → [57, 59]
[00022] NEW → [65]
[00024] NEW → [70, 73]
[00029] NEW → [83]
[00030] NEW → [85]
[00034] NEW → [93]
[00035] MISSING → [11]
[00036] NEW → [100]
[00037] MISSING → [13]
[00041] NEW → [112]
[00044] NEW → [123]
[00045] NEW → [128, 125]
[00046] MISSING → [16, 31]
[00049] MISSING → [34]
[00050] NEW → [130, 131]
[00052] NEW → [133]
[00054] NEW → [134]
[00054] MISSING → [55]
[00055] NEW → [135]
[00058] NEW → [137]
[00059] NEW → [

In [18]:
from pathlib import Path
import shutil, cv2, tqdm

SRC = Path("/kaggle/input/data-set-s/MOT20Det")          # read-only Kaggle dir
DST = Path("/kaggle/working/mot20_yolo")                 # writeable output
(DST/"images/train").mkdir(parents=True, exist_ok=True)
(DST/"images/val").mkdir(parents=True, exist_ok=True)
(DST/"labels/train").mkdir(parents=True, exist_ok=True)
(DST/"labels/val").mkdir(parents=True, exist_ok=True)

# choose one sequence for validation (here MOT20-04); you can change this split
VAL_SEQ = "MOT20-04"

def mot_to_yolo_line(x, y, w, h, img_w, img_h, cls=0):
    # convert absolute box to YOLO (cx,cy,w,h) normalized 0-1
    cx = (x + w/2) / img_w
    cy = (y + h/2) / img_h
    return f"{cls} {cx:.6f} {cy:.6f} {w/img_w:.6f} {h/img_h:.6f}\n"

def process_sequence(seq_path, split):
    imgs_out   = DST/f"images/{split}"
    labels_out = DST/f"labels/{split}"

    # load gt.txt into dict: frame_idx -> list of boxes
    gt_file = seq_path/"gt/gt.txt"
    boxes_per_frame = {}
    with open(gt_file) as f:
        for line in f:
            f_id, obj_id, x, y, w, h, conf, cls, vis = map(float, line.split(','))
            if int(cls) != 1:        # only 'person'
                continue
            boxes_per_frame.setdefault(int(f_id), []).append((x, y, w, h))

    # iterate images
    img_dir = seq_path/"img1"
    for img_path in sorted(img_dir.glob("*.jpg")):
        frame_id = int(img_path.stem)
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]

        # write label file
        label_lines = []
        for (x, y, bw, bh) in boxes_per_frame.get(frame_id, []):
            label_lines.append(mot_to_yolo_line(x, y, bw, bh, w, h))
        (labels_out/f"{img_path.stem}.txt").write_text("".join(label_lines))

        # copy image
        shutil.copy2(img_path, imgs_out/img_path.name)

# ---------- run over all sequences ----------
for seq in sorted((SRC/"train").iterdir()):
    split = "val" if seq.name == VAL_SEQ else "train"
    print(f"✏️  {seq.name}  →  {split}")
    process_sequence(seq, split)

print("✅  MOT20 conversion finished.")


✏️  MOT20-01  →  train
✏️  MOT20-02  →  train
✏️  MOT20-03  →  train
✏️  MOT20-05  →  train
✅  MOT20 conversion finished.


In [19]:
%%bash
cat > /kaggle/working/mot20.yaml <<'EOF'
path: /kaggle/working/mot20_yolo
train: images/train
val: images/val

nc: 1
names: ['person']
EOF
echo "✅  mot20.yaml written."


✅  mot20.yaml written.


In [21]:
# ──────────────────────────────────────────────────────────────────────────
# Split MOT20 YOLO folder into train / val
# root layout BEFORE running this cell:
#   mot20_yolo/
#      images/train/*.jpg
#      labels/train/*.txt
# AFTER running:
#      images/val/*.jpg
#      labels/val/*.txt
# -------------------------------------------------------------------------
from pathlib import Path
import random, shutil, yaml, math

# CONFIG ───────────────────────────────────────────────────────────────────
ROOT       = Path("/kaggle/working/mot20_yolo")  # change if you saved elsewhere
VAL_RATIO  = 0.20        # 10 % validation
RAND_SEED  = 42          # reproducible split
# ──────────────────────────────────────────────────────────────────────────

random.seed(RAND_SEED)

imgs_train_dir  = ROOT/"images/train"
labels_train_dir= ROOT/"labels/train"
imgs_val_dir    = ROOT/"images/val";    imgs_val_dir.mkdir(parents=True, exist_ok=True)
labels_val_dir  = ROOT/"labels/val";    labels_val_dir.mkdir(parents=True, exist_ok=True)

# 1️⃣  pick random subset
img_paths = sorted(imgs_train_dir.glob("*.jpg"))
n_val     = max(1, math.floor(len(img_paths)*VAL_RATIO))
val_imgs  = set(random.sample(img_paths, n_val))

# 2️⃣  move images + matching labels
for img_path in img_paths:
    stem = img_path.stem
    lbl_path = labels_train_dir/f"{stem}.txt"
    if img_path in val_imgs:
        shutil.move(img_path, imgs_val_dir/img_path.name)
        shutil.move(lbl_path, labels_val_dir/lbl_path.name)

print(f"✅  Moved {n_val} images to validation split "
      f"({len(img_paths)-n_val} remain in training).")

# 3️⃣  rewrite YAML so Ultralytics knows the paths
yaml_path = ROOT.parent/"mot20.yaml"   # e.g. /kaggle/working/mot20.yaml
data = dict(
    path=str(ROOT),
    train="images/train",
    val="images/val",
    nc=1,
    names=["person"]
)
yaml_path.write_text(yaml.dump(data, sort_keys=False))
print(f"✅  Updated {yaml_path}")

# quick sanity check
print("\nSample counts:")
print(" train images:", len(list((ROOT/'images/train').glob('*.jpg'))))
print(" val images  :", len(list((ROOT/'images/val').glob('*.jpg'))))


✅  Moved 663 images to validation split (2652 remain in training).
✅  Updated /kaggle/working/mot20.yaml

Sample counts:
 train images: 2652
 val images  : 663


In [22]:
!yolo train model=yolov8n.pt data=/kaggle/working/mot20.yaml \
            epochs=40 imgsz=800 batch=16 device=0


Ultralytics 8.3.119 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/mot20.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=No

In [23]:
# One-time inside a new code-cell
!pip install --quiet python-docx gitpython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.5 MB/s eta 0:00:00a 0:00:01


In [24]:
import shutil, pathlib, json, subprocess, os, cv2, time, torch
from ultralytics import YOLO

ROOT = pathlib.Path('/kaggle/working')
RUNS = ROOT/'runs/detect'          # Ultralytics default
BEST = next((RUNS.glob('train*/weights/best.pt'))).__str__()  # first best.pt

# Make project folder
PRJ = ROOT/'mot20_rt'
(PRJ/'src').mkdir(parents=True, exist_ok=True)
(PRJ/'assets').mkdir(exist_ok=True)

shutil.copy2(BEST, PRJ/'yolov8_mot20.pt')
print("✅ copied weight to:", PRJ/'yolov8_mot20.pt')


✅ copied weight to: /kaggle/working/mot20_rt/yolov8_mot20.pt


In [25]:
code = r"""
import argparse, cv2, time, pathlib, torch
from ultralytics import YOLO

def main(opt):
    device = 0 if torch.cuda.is_available() else 'cpu'
    model  = YOLO(opt.weights).fuse().to(device)
    if device != 'cpu': model.model.half()

    cap = cv2.VideoCapture(opt.source)
    fps_in = cap.get(cv2.CAP_PROP_FPS) or 25
    w, h = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter(opt.out, cv2.VideoWriter_fourcc(*'mp4v'), fps_in, (w, h))

    t0, frame = time.time(), 0
    while True:
        ok, im = cap.read()
        if not ok: break
        res = model.track(im, tracker="bytetrack.yaml", persist=True, verbose=False)
        out.write(res[0].plot())
        frame += 1
    cap.release(); out.release()
    fps = frame/(time.time()-t0)
    pathlib.Path('assets/fps.txt').write_text(f"{fps:.2f}\\n")
    print(f"Processed {frame} frames → {fps:.2f} FPS")
if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument('--source', default='demo.mp4')
    p.add_argument('--weights', default='yolov8_mot20.pt')
    p.add_argument('--out', default='assets/output_demo.mp4')
    main(p.parse_args())
"""
(PRJ/'src/rt_detect.py').write_text(code)
print("✅ inference script saved.")


✅ inference script saved.


In [37]:
from pathlib import Path
import shutil

PRJ = Path("/kaggle/working/mot20_rt")   # folder that will hold your repo
PRJ.mkdir(parents=True, exist_ok=True)
print("PROJECT DIR:", PRJ)


PROJECT DIR: /kaggle/working/mot20_rt


In [38]:
# copy best.pt only if it isn't there yet
BEST = next(Path("runs/detect").glob("*/weights/best.pt"), None)
assert BEST, "❌ best.pt not found under runs/detect"
shutil.copy2(BEST, PRJ/"yolov8_mot20.pt")
print("✅ weight copied to", PRJ/"yolov8_mot20.pt")


✅ weight copied to /kaggle/working/mot20_rt/yolov8_mot20.pt


In [39]:
code = r"""
import argparse, cv2, time, pathlib, torch, sys
from ultralytics import YOLO

def main():
    p = argparse.ArgumentParser()
    p.add_argument('--source',  default='demo.mp4')
    p.add_argument('--weights', default='yolov8_mot20.pt')
    p.add_argument('--out',     default='assets/output_demo.mp4')
    # accept and ignore any unknown args Jupyter may tack on
    opt, _ = p.parse_known_args()

    device = 0 if torch.cuda.is_available() else 'cpu'

    model = YOLO(opt.weights)
    model.fuse()                 # in-place
    model.to(device)
    if device != 'cpu':
        model.model.half()

    cap = cv2.VideoCapture(opt.source)
    if not cap.isOpened():
        sys.exit(f"❌ cannot open video {opt.source}")
    fps_in = cap.get(cv2.CAP_PROP_FPS) or 25
    w, h = int(cap.get(3)), int(cap.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    pathlib.Path(opt.out).parent.mkdir(parents=True, exist_ok=True)
    out = cv2.VideoWriter(opt.out, fourcc, fps_in, (w, h))

    t0, frames = time.time(), 0
    while True:
        ok, img = cap.read()
        if not ok:
            break
        res = model.track(img, tracker="bytetrack.yaml", persist=True, verbose=False)
        out.write(res[0].plot())
        frames += 1

    cap.release(); out.release()
    fps = frames / max(1e-6, (time.time() - t0))
    pathlib.Path("assets").mkdir(exist_ok=True)
    pathlib.Path("assets/fps.txt").write_text(f"{fps:.2f}\n")
    print(f"Processed {frames} frames  →  {fps:.2f} FPS")

if __name__ == "__main__":
    main()
"""
(PRJ/'src').mkdir(parents=True, exist_ok=True)
(PRJ/'src/rt_detect.py').write_text(code)
print("✅ script written:", PRJ/'src/rt_detect.py')


✅ script written: /kaggle/working/mot20_rt/src/rt_detect.py


In [42]:
import os, shutil
from pathlib import Path

PRJ = Path("/kaggle/working/mot20_rt")
PRJ.mkdir(parents=True, exist_ok=True)

# make it visible to subsequent %%bash cells
os.environ["PRJ"] = str(PRJ)
print("PROJECT DIR:", PRJ)


PROJECT DIR: /kaggle/working/mot20_rt


In [44]:
demo_mp4 = PRJ / "demo.mp4"

cmd = [
    "ffmpeg", "-y", "-hide_banner", "-loglevel", "error",
    "-framerate", "25",
    "-i", "/kaggle/input/data-set-s/MOT20Det/test/MOT20-04/img1/%06d.jpg",
    "-t", "10",
    "-vf", "format=yuv420p,scale=trunc(iw/2)*2:trunc(ih/2)*2",
    str(demo_mp4)
]
print("Running FFmpeg …")
subprocess.run(cmd, check=True)
print("✅ demo.mp4 ready:", demo_mp4)


Running FFmpeg …
✅ demo.mp4 ready: /kaggle/working/mot20_rt/demo.mp4


In [45]:
# make sure the weight file exists (copy once if needed)
best = next(Path("runs/detect").glob("*/weights/best.pt"), None)
assert best, "best.pt not found"
shutil.copy2(best, PRJ/"yolov8_mot20.pt")

# call your script
subprocess.run([
    "python", str(PRJ/"src/rt_detect.py"),
    "--source", str(demo_mp4),
    "--weights", str(PRJ/"yolov8_mot20.pt"),
    "--out", str(PRJ/"assets/output_demo.mp4")
], check=True)

# extract one still for the report
subprocess.run([
    "ffmpeg", "-y", "-hide_banner", "-loglevel", "error",
    "-i", str(PRJ/"assets/output_demo.mp4"),
    "-vframes", "1", str(PRJ/"assets/sample_frame.jpg")
], check=True)

print("✅ all artifacts created in", PRJ/"assets")


Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
Processed 250 frames  →  7.80 FPS
✅ all artifacts created in /kaggle/working/mot20_rt/assets


In [47]:
from pathlib import Path
import argparse, cv2, time, torch
from ultralytics import YOLO
import collections

def main():
    p = argparse.ArgumentParser()
    p.add_argument("--source",  default="demo.mp4")
    p.add_argument("--weights", default="/kaggle/working/mot20_rt/yolov8_mot20.pt")
    p.add_argument("--out",     default="assets/output_events.mp4")
    p.add_argument("--miss_tol",type=int, default=30,
                   help="frames before an ID is marked MISSING")
    opt, _ = p.parse_known_args()

    device = 0 if torch.cuda.is_available() else "cpu"
    model  = YOLO(opt.weights)
    model.fuse(); model.to(device)
    if device != "cpu":
        model.model.half()

    # ── NEW / MISSING state holder ───────────────────────────────────────
    last_seen = {}
    frame_idx = 0

    cap = cv2.VideoCapture(opt.source)
    fps_in = cap.get(cv2.CAP_PROP_FPS) or 25
    w, h   = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter(opt.out,
                          cv2.VideoWriter_fourcc(*"mp4v"),
                          fps_in, (w, h))

    t0, frames = time.time(), 0
    while True:
        ok, img = cap.read()
        if not ok:
            break

        res = model.track(img, tracker="bytetrack.yaml",
                          persist=True, verbose=False)
        tracks = res[0].boxes

        # ---- event logic -------------------------------------------------
        current_ids = {int(t.id) for t in tracks if t.id is not None}
        new_ids     = [tid for tid in current_ids if tid not in last_seen]
        missing_ids = [tid for tid, last in list(last_seen.items())
                       if frame_idx - last > opt.miss_tol]

        # update last_seen
        for tid in current_ids:
            last_seen[tid] = frame_idx
        for tid in missing_ids:
            last_seen.pop(tid, None)

        frame_idx += 1
        frames += 1

        canvas = res[0].plot()   # boxes+IDs
        # draw event text top-left
        txt = f"NEW:{new_ids}  MISSING:{missing_ids}"
        cv2.putText(canvas, txt, (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                    (0,255,0), 2, cv2.LINE_AA)

        out.write(canvas)

    cap.release(); out.release()
    fps = frames / max(1e-6, time.time()-t0)
    Path("assets").mkdir(exist_ok=True)
    Path("assets/fps.txt").write_text(f"{fps:.2f}\n")
    print(f"Processed {frames} frames → {fps:.2f} FPS")

if __name__ == "__main__":
    main()


Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
Processed 0 frames → 0.00 FPS


In [49]:
from pathlib import Path
import os

PRJ = Path("/kaggle/working/mot20_rt")          # project root
(PRJ / "src").mkdir(parents=True, exist_ok=True)

code = r"""
import argparse, cv2, time, torch, pathlib, sys
from ultralytics import YOLO

def main():
    p = argparse.ArgumentParser()
    p.add_argument("--source",  required=True)
    p.add_argument("--weights", required=True)
    p.add_argument("--out",     required=True)
    p.add_argument("--miss_tol",type=int, default=30)
    opt, _ = p.parse_known_args()

    device = 0 if torch.cuda.is_available() else "cpu"
    model  = YOLO(opt.weights)
    model.fuse(); model.to(device)
    if device != "cpu": model.model.half()

    last_seen, frame_id = {}, 0
    cap = cv2.VideoCapture(opt.source)
    if not cap.isOpened():
        sys.exit(f"❌ cannot open {opt.source}")
    fps_in = cap.get(cv2.CAP_PROP_FPS) or 25
    w, h   = int(cap.get(3)), int(cap.get(4))
    out_v  = cv2.VideoWriter(opt.out,
              cv2.VideoWriter_fourcc(*"mp4v"), fps_in, (w, h))

    t0 = time.time()
    while True:
        ok, img = cap.read()
        if not ok: break
        res = model.track(img, tracker="bytetrack.yaml", persist=True, verbose=False)
        ids = {int(b.id) for b in res[0].boxes if b.id is not None}

        new_ids  = [i for i in ids if i not in last_seen]
        gone_ids = [i for i, last in list(last_seen.items())
                    if frame_id - last > opt.miss_tol]

        for i in ids: last_seen[i] = frame_id
        for i in gone_ids: last_seen.pop(i, None)
        frame_id += 1

        frame = res[0].plot()
        txt = f"NEW:{new_ids}  MISSING:{gone_ids}"
        cv2.putText(frame, txt, (10,30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0,255,0), 2, cv2.LINE_AA)
        out_v.write(frame)

    cap.release(); out_v.release()
    fps = frame_id / max(1e-6, time.time()-t0)
    pathlib.Path("assets").mkdir(exist_ok=True)
    pathlib.Path("assets/fps.txt").write_text(f"{fps:.2f}\n")
    print(f"Done – {frame_id} frames → {fps:.2f} FPS")

if __name__ == "__main__":
    main()
"""
(PRJ/"src/rt_detect_events.py").write_text(code)
print("✅  rt_detect_events.py created at", PRJ/"src/rt_detect_events.py")


✅  rt_detect_events.py created at /kaggle/working/mot20_rt/src/rt_detect_events.py


In [51]:
import subprocess, os, shutil
from pathlib import Path

PRJ = Path("/kaggle/working/mot20_rt")

# ensure weight is present
best = next(Path("runs/detect").glob("*/weights/best.pt"), None)
assert best, "best.pt not found"
shutil.copy2(best, PRJ/"yolov8_mot20.pt")

# make 15-s clip from unseen TEST split
test_clip = PRJ/"demo_test.mp4"
subprocess.run([
    "ffmpeg","-y","-hide_banner","-loglevel","error",
    "-framerate","25",
    "-i","/kaggle/input/data-set-s/MOT20Det/test/MOT20-04/img1/%06d.jpg",
    "-t","15",
    "-vf","format=yuv420p,scale=trunc(iw/2)*2:trunc(ih/2)*2",
    str(test_clip)
], check=True)

# run detector with NEW/MISSING overlay
subprocess.run([
    "python", str(PRJ/"src/rt_detect_events.py"),
    "--source",  str(test_clip),
    "--weights", str(PRJ/"yolov8_mot20.pt"),
    "--out",     str(PRJ/"assets/output_events.mp4")
], check=True)

# grab one still
subprocess.run([
    "ffmpeg","-y","-hide_banner","-loglevel","error",
    "-i", str(PRJ/"assets/output_events.mp4"),
    "-vframes","1", str(PRJ/"assets/sample_events.jpg")
], check=True)

print("\n🎉  All artifacts ready in", PRJ/"assets")
print("    ->", list((PRJ/"assets").iterdir()))


Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
Done – 375 frames → 8.15 FPS

🎉  All artifacts ready in /kaggle/working/mot20_rt/assets
    -> [PosixPath('/kaggle/working/mot20_rt/assets/output_demo.mp4'), PosixPath('/kaggle/working/mot20_rt/assets/sample_frame.jpg'), PosixPath('/kaggle/working/mot20_rt/assets/sample_events.jpg'), PosixPath('/kaggle/working/mot20_rt/assets/output_events.mp4')]
